In [1]:
def forward_kinematics(theta1, theta2, l1, l2):
    '''
    Forward kinematics module for a serial-2R chain.
    The base of the manipulator is assumed to be placed at the
    coordinates [0,0].
    All the joints allow rotation about the positive Z-axis.
    Args:
    --- theta1: Angle between the link l1 and the positive x-axis (in radians)
    --- theta2: Relative angle between link l1 and link l2 (in radians)
    --- l1: Length of link l1 (in m)
    --- l2: Length of link l2 (in m)
    Ret:
    --- [x, y]: Position co-ordinates of the end-effector (in m)
    '''
    x = l1*math.cos(theta1) + l2*math.cos(theta1 + theta2)
    y = l1*math.sin(theta1) + l2*math.sin(theta1 + theta2)
    return [x, y]

def inverse_kinematics(x, y, l1, l2, branch=1):
    '''
    Inverse kinematics modules for the serial-2R manipulator.
    The base of the manipulator is placed at [0,0].
    Axis of rotation is the Z+ axis.
    Args:
    --- x : X co-ordinate of the end-effector
    --- y : Y co-ordinate of the end-effector
    --- l1: Length of link l1
    --- l2: Length of link l2
    --- branch: Branch of the inverse kinematics solution.
    Ret:
    --- valid: Binary variable indicating if the solution is valid or not
    --- [theta1, theta2]: Angles made by link l1 w.r.t X+ axis and the relative
                    angle between links l1 and l2 respectively.
    '''
    a = 2*x*l2
    b = 2*y*l2
    c =  l1*l1 - x*x - y*y  - l2*l2 
    psi = math.atan2(b, a)
    d = -c/math.sqrt(a*a + b*b)
    
    if (d < -1) or (d > 1):
        print("Position out of workspace.")
        return False, [0,0]
    if branch == 1:
        theta12 = psi + math.acos(-c/math.sqrt(a*a + b*b))
    else:
        theta12 = psi - math.acos(-c/math.sqrt(a*a + b*b))
        
    theta1 = math.atan2((y - l2*math.sin(theta12))/l1, (x - l2*math.cos(theta12))/l1)
    return True, [theta1, theta12-theta1]

In [2]:
''' 
Verification.

Verify the correctness of the FK and IK modules by 
checking them against each other.
'''

import math
import numpy

[x, y] = forward_kinematics(math.pi/2, 0, 1, 2)
valid, [theta1, theta2] = inverse_kinematics(0, -3.0, 1, 2)


l1 = 1
l2 = 2

for _ in range(1000):
    theta1 = numpy.random.rand()
    theta2 = numpy.random.rand()

    [x, y] = forward_kinematics(theta1, theta2, l1, l2)
    valid, [t1, t2] = inverse_kinematics(x, y, l1, l2)

    print(t1 - theta1, t2 - theta2)

1.8369701987210297e-16 3.0
-1.5707963267948968 2.220446049250313e-16
4.996003610813204e-16 -8.326672684688674e-16
-2.220446049250313e-16 4.440892098500626e-16
0.0 1.1102230246251565e-16
4.3021142204224816e-16 -7.771561172376096e-16
-3.1086244689504383e-15 4.6629367034256575e-15
2.220446049250313e-16 -4.440892098500626e-16
1.2212453270876722e-15 -1.887379141862766e-15
2.7755575615628914e-16 -5.551115123125783e-16
4.163336342344337e-17 0.0
-3.3306690738754696e-16 5.551115123125783e-16
-6.661338147750939e-16 1.1102230246251565e-15
-2.7755575615628914e-17 -1.1102230246251565e-16
5.204170427930421e-16 -8.881784197001252e-16
-1.6653345369377348e-15 2.55351295663786e-15
-2.220446049250313e-16 1.1102230246251565e-16
4.440892098500626e-16 -6.661338147750939e-16
1.1102230246251565e-16 -1.1102230246251565e-16
-4.440892098500626e-16 7.771561172376096e-16
-4.440892098500626e-16 7.771561172376096e-16
2.4424906541753444e-15 -3.774758283725532e-15
-1.8041124150158794e-16 4.440892098500626e-16
-6.82787